In [73]:
# Custom functionalities
import visionNew
from Utilities.metrics import pixelwiseAccuracy
from Utilities.datasetHandler import CityscapesDataset
from Utilities.modelHandler import modelFCN

import torch
from torchvision.io.image import read_image
from torchvision.transforms.functional import to_pil_image
from torch.utils.data import DataLoader
from  torch import nn
from torchvision.models import ResNet50_Weights
from torchvision.models.segmentation import fcn_resnet50, FCN_ResNet50_Weights
from torch.nn import CrossEntropyLoss
from torch.optim import Adam

device = torch.device('cpu')
if torch.cuda.is_available(): device = torch.device('cuda')

torch.cuda.empty_cache()

## Test dataset

# Read single image
dtTEST = CityscapesDataset(image_directory="TestPascal", phase=None, num_classes=30)


gt = read_image("aachen_000000_000019_gtFine_labelIds.png")      
#img = dtTEST.get_ground_truth_path("berlin_000000_000019_gtFine_labelIds.png")

In [74]:
### DATASET ###
img_directory = '/mnt/data/course/psarin/inm705/leftImg8bit'
ground_truth_directory = '/mnt/data/course/psarin/inm705/gtFine_trainvaltest/gtFine'
n_classes = 30

# Import datasets
dt_train = CityscapesDataset(image_directory = img_directory,
                            gt_directory = ground_truth_directory,
                            phase = 'train',
                            num_classes = n_classes)

dt_val = CityscapesDataset(image_directory = img_directory,
                            gt_directory = ground_truth_directory,
                            phase = 'val',
                            num_classes = n_classes)

dt_test = CityscapesDataset(image_directory = img_directory,
                            gt_directory = ground_truth_directory,
                            phase = 'test',
                            num_classes = n_classes)

# Initialise dataloaders
dataloader_args = {'batch_size':20, 'shuffle':False}

## TRAIN dataset
trainloader = DataLoader(dt_train, **dataloader_args)
## VAL dataset
valloader = DataLoader(dt_val, **dataloader_args)
## TEST dataset
testloader = DataLoader(dt_test, batch_size=10)


# Pascal VOC categories
object_categories = ['aeroplane', 'bicycle', 'bird', 'boat',
                     'bottle', 'bus', 'car', 'cat', 'chair',
                      'cow', 'diningtable', 'dog', 'horse',
                      'motorbike', 'person', 'pottedplant',
                     'sheep', 'sofa', 'train', 'tvmonitor']
#weights.meta["categories"]


# convert list to dict
pascal_voc_classes = {}
for id, name in enumerate(object_categories): 
    pascal_voc_classes[name]=id # these are the same indices used to create the label vector y in the dataset 
    

['aachen']
['frankfurt']
['berlin']


In [75]:
## MODEL Pretrained with RESNET 50
# Initialize model with the best available weights
FCN_weights = FCN_ResNet50_Weights.DEFAULT
backbone_weights = ResNet50_Weights.DEFAULT
model = fcn_resnet50(backbone_weights=backbone_weights).to(device)

model.backbone.requires_grad_(False)
model.classifier[4] = nn.Conv2d(512, 30, kernel_size=(1, 1), stride=(1, 1))
model.classifier.requires_grad_(True)


FCNHead(
  (0): Conv2d(2048, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Dropout(p=0.1, inplace=False)
  (4): Conv2d(512, 30, kernel_size=(1, 1), stride=(1, 1))
)

In [76]:
cel = CrossEntropyLoss()
ad = Adam(model.parameters())

fcn1 = modelFCN(model, loss_function=cel, optimizer = ad )

In [77]:
#L = fcn1.train_model(trainloader, total_epochs=4)

In [78]:
# Load checkpoint
#fcn1.model.load_state_dict(torch.load("Checkpoints/test_epoch_0.pth"))
fcn1.model.load_state_dict(torch.load("city_test_epoch_3.pth"))#.
fcn1.model.eval();

In [79]:
# Step 4: Use the model and visualize the prediction
_, X_test, y_test = next(iter(testloader))
X_test, y_test = X_test.to(device), y_test.to(device) 

fcn1.model.to(device)
pred = fcn1.model(X_test)['out']

In [80]:
out = torch.sigmoid(pred) > 0.5 # probability tensor for 0 vs 1 separatation threshold
out = out.int()

In [81]:
# Step 4: Use the model and visualize the prediction
#prediction = pred["out"]
#pred_aux = pred["aux"]


#normalized_masks = pred.softmax(dim=1)
#normalized_aux = pred_aux.softmax(dim=1)

# Indices of classes
#class_to_idx = {cls: idx for (idx, cls) in enumerate(weights.meta["categories"])}

#n_lab = 2


#mask = normalized_masks[0, 3]#class_to_idx["sofa"]] #+ normalized_masks[0, class_to_idx["horse"]]
#to_pil_image(mask)

#print(mask.size())


In [82]:
pixelwiseAccuracy(out,y_test)

0.9891991019248962

## Check the model

In [83]:
from Utilities.torchsummaryNew import summary

In [84]:
device = torch.device('cuda')

In [85]:
x = torch.randn(1, 3, 224, 224).to(device)
fcn1.model.to(device);

In [86]:
summary(fcn1.model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

(tensor(32961630), tensor(9453598))